In [2]:
!pip install -r ../requirements.txt

In [5]:
!git clone https://github.com/danibachar/idc_CANLab.git

Cloning into 'idc_CANLab'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 17 (delta 2), reused 12 (delta 1), pack-reused 0
Unpacking objects: 100% (17/17), done.


In [7]:
!pwd

/Users/danielbachar/Documents/IDC/idc_CANLab_New/notebooks


In [8]:
import idc_CANLab.functions # Our functionalities

from ipywidgets import interact, interactive, fixed, interact_manual, FileUpload
import ipywidgets as widgets
from IPython.display import display

import os 
import io
import time
import json
# Data manipulation and calculation packages
import pandas as pd
import numpy as np
### Visualization
# Matplotlib
import matplotlib.pyplot as plt
# Bokeh
import bokeh.plotting
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, CDSView, GroupFilter
from bokeh.io import output_notebook
# Call once to configure Bokeh to display plots inline in the notebook.
output_notebook()

# this will allow the notebook to reload/refresh automatically within the runtime
%reload_ext autoreload
%autoreload 2

# File Uploads
## Please note we currently support only CSV files.
##    If your'e using Excel or other file formats please convert to CSV
###  Excel to CSV - https://knowledgebase.constantcontact.com/articles/KnowledgeBase/6409-saving-an-excel-file-as-a-csv-file?lang=en_US
### EDF to CSV - https://emotiv.gitbook.io/emotivpro/convert_edf_to_csv